In [2]:
# dados fin
import yfinance as yf

start = '2021-01-01'
end = '2023-01-01'

data = yf.download('AAPL', 
                   start=start,
                   end=end)

data.head()

data.to_csv('data_/AAPL.csv')

[*********************100%***********************]  1 of 1 completed


In [4]:
import pandas as pd

# Carregando os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,133.520004,133.610001,126.760002,129.410004,127.503639,143301900
1,2021-01-05,128.889999,131.740005,128.429993,131.009995,129.080048,97664900
2,2021-01-06,127.720001,131.050003,126.379997,126.599998,124.735023,155088000
3,2021-01-07,128.360001,131.630005,127.860001,130.919998,128.991379,109578200
4,2021-01-08,132.429993,132.630005,130.229996,132.050003,130.104752,105158200


In [5]:
def calculate_moving_average(data, window):
    return data['Close'].rolling(window=window).mean()

# Vamos testar para diferentes períodos de média móvel (por exemplo, de 5 a 50 dias)
moving_averages = []
for window in range(5, 51):
    moving_averages.append(calculate_moving_average(data, window))


In [6]:
def calculate_distance_from_moving_average(data, moving_average):
    return abs(data['Close'] - moving_average)

distance_from_moving_averages = []
for moving_average in moving_averages:
    distance_from_moving_averages.append(calculate_distance_from_moving_average(data, moving_average))

# Vamos calcular a média das distâncias para cada período
mean_distances = [dist.mean() for dist in distance_from_moving_averages]


In [7]:
best_window = mean_distances.index(min(mean_distances)) + 5  # +5 pois começamos a partir da janela 5

print(f'A melhor média móvel é de {best_window} dias.')

A melhor média móvel é de 5 dias.


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LinearRegression

# Carregando os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Definindo a função para calcular a distância da média móvel
def calculate_distance_from_moving_average(data, window):
    moving_average = data['Close'].rolling(window=window).mean()
    return abs(data['Close'] - moving_average)

# Criação de um array de períodos de média móvel a serem testados
window_sizes = np.arange(5, 51)

# Preparação dos dados para validação cruzada
X = data[['Close']]
y = data['Close']

# Definindo o modelo de regressão linear
model = LinearRegression()

# Definindo os parâmetros a serem testados no GridSearch
param_grid = {'window': window_sizes}

# Definindo a estratégia de validação cruzada
tscv = TimeSeriesSplit(n_splits=5)

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Executando a busca pelo melhor período de média móvel
grid_search.fit(X, y)

# Extraindo os resultados
best_window = grid_search.best_params_['window']
best_mse = -grid_search.best_score_

print(f"A melhor média móvel é de {best_window} dias, com MSE de {best_mse}.")